In [133]:
import numpy as np
import pandas as pd

In [134]:
fname = '가 나 다 라 마 바 사 아 자 차 카 타 파 하 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split()

In [135]:
df = pd.DataFrame()
for fn in fname:
    try:
        tmp = pd.read_csv(f'../static/data/movie{fn}.csv')
    except:
        continue
    df = pd.concat([df, tmp])
df.shape

(3646, 10)

In [136]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\n사랑하는 사..."


In [137]:
df.set_index('code', inplace=True)
df.reset_index(inplace=True)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)


(3646, 10)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\n사랑하는 사..."


In [138]:
df.isna().sum()

code               0
title              0
genre              0
summering          0
first_day          0
production_year    0
movie_director     0
star_actor         0
img                0
synopsis           0
dtype: int64

In [139]:
m_type, m_kind, m_genre, m_time, m_rated, m_nation = [], [], [], [], [], []
for info in df.summering.values:
    infos = info.split('|')
    infos_len = len(infos)
    m_type.append(infos[0].strip())                     # 장편
    m_kind.append(infos[1].strip())                     # 일반영화, 예술영화..
    m_genre.append(infos[2].strip().replace(',', ''))   # 애니메이션, 드라마, 멜로
    m_nation.append(infos[-1].strip())                  # 한국
    
    # 요약 정보 개수가 다를 수 있다
    if infos_len == 6:
        m_time.append(infos[-3].strip())                # 70분
        m_rated.append(infos[-2].strip())
    elif infos_len == 5 :
        m_time.append(infos[-2].strip())                # 12세 이상 관람가
        m_rated.append('')
    elif infos_len == 4:
        m_rated.append('')
        m_time.append('')
    else:
        print(info)

len(m_type)

3646

In [140]:
df['m_type'] = m_type
df['m_kind'] = m_kind
df['m_genre'] = m_genre
df['m_nation'] = m_nation
df['m_time'] = m_time
df['m_rated'] = m_rated

In [105]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\n사랑하는 사...",장편,예술영화,SF,프랑스,97분 57초,12세이상관람가


In [ ]:
df.tail(5)

In [141]:
indexs = df[df.summering.str.contains('멜로/로맨스') & df.summering.str.contains('청소년관람불가')].index

In [61]:

df[df.star_actor.str.find('[') >= 0]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated


In [160]:
# 감독 자리에 다른 값들이 들어갔다. 주연 배우 자리에는 데이터가 없고
# 잘못 처리했다. 일단 감독이 이상한 것은 지우고 테스트
indexs = df[df.movie_director.str.find('[') >= 0].index
indexs

Index([  33,   39,  122,  125,  126,  127,  128,  129,  130,  131,
       ...
       2383, 2401, 2408, 2411, 2420, 2449, 2468, 2568, 2606, 2642],
      dtype='int64', length=207)

In [142]:
df.drop(df.index[indexs], inplace=True)
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [143]:
df.shape

(2746, 16)

In [84]:
for index in df[df.movie_director.str.find('[') >= 0].index:
    df.iloc[index, 6] = ''

##### 시놉시스 형태소 분석해 놓기

In [144]:
from konlpy.tag import Okt
okt = Okt()
stop_words = '''의 가 에 과 을 의 를 는 이 은 한편 와 
            '''.split()

In [145]:
import re, string

morphs = []

for text in df.synopsis:
    text = re.sub('['+string.punctuation+'\r\n“”‘’…]', ' ', text)
    tokens = okt.morphs(text)
    morphs.append(' '.join([word for word in tokens if word not in stop_words]))

df['morphs'] = morphs


In [129]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\n사랑하는 사...",장편,예술영화,SF,프랑스,97분 57초,12세이상관람가,가가린 옥상 에서 하늘 보며 우주비행사 꿈꾸던 10 대 소년 유리 사랑 하는 사람 ...


In [146]:
df.to_csv('../static/data/movie_test.csv', index=False)

In [ ]:
for title in df.title:
    print(title)